In [54]:
import torch;
import torch.nn as nn;
import torch.optim as optim;
import torch.nn.functional as F;
from torch.utils.data import DataLoader;
import torchvision.datasets as datasets;
import torchvision.transforms as transforms;

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

print(device, torch.__version__)

mps 2.4.1


In [52]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [58]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 1

train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [66]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [67]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)

        print(epoch, batch_idx, data.shape, targets.shape)

        data = data.reshape(data.shape[0], -1)

        print(epoch, batch_idx, data.shape, targets.shape)

        scores = model(data)
        loss = criterion(scores, targets)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()


0 0 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 0 torch.Size([64, 784]) torch.Size([64])
0 1 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 1 torch.Size([64, 784]) torch.Size([64])
0 2 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 2 torch.Size([64, 784]) torch.Size([64])
0 3 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 3 torch.Size([64, 784]) torch.Size([64])
0 4 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 4 torch.Size([64, 784]) torch.Size([64])
0 5 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 5 torch.Size([64, 784]) torch.Size([64])
0 6 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 6 torch.Size([64, 784]) torch.Size([64])
0 7 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 7 torch.Size([64, 784]) torch.Size([64])
0 8 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 8 torch.Size([64, 784]) torch.Size([64])
0 9 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 9 torch.Size([64, 784]) torch.Size([64])
0 10 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 10 torch.Size([64, 784]) tor

In [70]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0

    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f'{num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

    model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

55773/60000 with accuracy 92.95
9298/10000 with accuracy 92.98
